In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

Using TensorFlow backend.


In [2]:
A, X, E, y = qm9.load_data(return_type='numpy',
                          nf_keys='atomic_num',
                          ef_keys='type',
                          self_loops=True,
                          amount=4000)
uniq_X = np.unique(X)
X = label_to_one_hot(X, uniq_X)

Loading QM9 dataset.
Reading SDF


100%|██████████| 4000/4000 [00:01<00:00, 2385.50it/s]


In [3]:
y

,mol_id,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,gdb_3996,3.33117,1.516440,1.358690,1.0785,68.31,-0.2459,0.0010,0.2469,986.2809,0.139181,-400.944193,-400.935445,-400.934501,-400.977158,31.893,-1565.545221,-1575.164307,-1584.647850,-1452.224626
3996,gdb_3997,3.21675,1.545420,1.343710,0.9421,68.25,-0.2486,0.0004,0.2490,983.7517,0.139515,-400.947904,-400.939105,-400.938161,-400.981336,32.281,-1567.873907,-1577.460990,-1586.944533,-1454.846359
3997,gdb_3998,3.70487,1.351230,0.996100,0.6382,70.97,-0.2459,-0.0376,0.2084,1091.3741,0.091451,-361.346440,-361.337818,-361.336874,-361.379798,30.819,-1327.810927,-1333.955495,-1341.067054,-1241.734889
3998,gdb_3999,3.70937,1.364910,1.003890,4.0964,65.92,-0.2614,-0.0540,0.2074,1060.7387,0.081305,-377.453740,-377.445525,-377.444580,-377.486899,28.613,-1246.538471,-1252.049255,-1258.567191,-1166.714934


In [4]:
task_learned, task_transferred = 'zpve', 'h298_atom'
y_learned = StandardScaler().fit_transform(y[[task_learned]].values).reshape(-1, y[[task_learned]].values.shape[-1])
y_transferred = StandardScaler().fit_transform(y[[task_transferred]].values).reshape(-1, y[[task_transferred]].values.shape[-1])

In [5]:
y_learned

array([[-2.26683475],
       [-2.59395249],
       [-3.00266865],
       ...,
       [-0.79661508],
       [-1.11602001],
       [-0.79415958]])

In [6]:
y_transferred

array([[4.03511509],
       [4.5364037 ],
       [4.80754546],
       ...,
       [0.12817109],
       [0.47104808],
       [0.2246248 ]])

In [7]:
N = X.shape[-2]
F = X.shape[-1]
S = E.shape[-1]
n_out = y_learned.shape[-1]
learning_rate = 1e-3
epochs = 3
batch_size = 64
es_patience = 5

In [10]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
y_learned_train, \
y_learned_test, \
y_trasnferred_train, \
y_transferred_test = train_test_split(A, X, E, y_learned, y_transferred, test_size=0.1)

In [11]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

In [12]:
def create_encoder(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')[[gc1, A_in, E_in]]
    pool = GlobalAttentionPool(256)(gc2)
    return Model(inputs=[X_in, A_in, E_in], outputs=pool)

In [13]:
def create_classifier(pool):
    dense = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense)
    return Model(inputs=pool, outputs=output)

In [15]:
learned_encoder = create_encoder(X_in, E_in, A_in)
learned_classifer = create_classifier()

learned_model = Model(input=learned_encoder.input, output=learned_classifier(learned_encoder.input))

ValueError: Tensor t0 of rank 6 does not match einsum reduction of length 5